In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mount data từ Google Drive và giải nén vào folder đích

In [4]:
%rm -rf /content/__MACOSX
%rm -rf /content/data
%mkdir /content/data

!unzip -qq /content/drive/MyDrive/captcha/debug-10k.zip
%mv /content/debug-10k /content/data/train
!echo "Done train Data"

!unzip -qq /content/drive/MyDrive/captcha/debug-2k.zip
%mv /content/debug-2k /content/data/validation
!echo "Done validation Data"

%rm -rf /content/__MACOSX

Done train Data
Done validation Data


Load dataset vào bộ nhớ để chia thành 2 tập train và validation

In [5]:
import tensorflow as tf
from tensorflow.keras import preprocessing

BATCH_SIZE = 128
COLOR_MODE = "rgba"
IMG_HEIGHT = 50
IMG_WIDTH = 180

train_ds = preprocessing.image_dataset_from_directory(
  directory="data/train",
  label_mode="categorical",
  seed=123,
  color_mode=COLOR_MODE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

validation_ds = preprocessing.image_dataset_from_directory(
  directory="data/validation",
  label_mode="categorical",
  seed=123,
  color_mode=COLOR_MODE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  batch_size=BATCH_SIZE)

# Configure the dataset for performance
def configure_for_performance(ds):
    ds = ds.cache()
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
validation_ds = configure_for_performance(validation_ds)

Found 45786 files belonging to 23 classes.
Found 9201 files belonging to 23 classes.


In [6]:
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import optimizers
from tensorflow.keras import Sequential

MODEL_OUTPUT_DIR="/content/drive/MyDrive/captcha/captcha-model"
CLASS_NO = 23
EPOCHS = 120

def get_model():
    m = Sequential([
      layers.experimental.preprocessing.Rescaling(1./255, name="rescaling"),

      layers.Conv2D(32, (3, 3), padding="same", input_shape=(IMG_HEIGHT, IMG_WIDTH, 1)),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(64, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(128, (3, 3), padding="same"),
      layers.Conv2D(64, (1, 1), padding="same"),
      layers.Conv2D(128, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(256, (3, 3), padding="same"),
      layers.Conv2D(128, (1, 1), padding="same"),
      layers.Conv2D(256, (3, 3), padding="same"),
      layers.Conv2D(128, (1, 1), padding="same"),
      layers.Conv2D(256, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.Conv2D(256, (1, 1), padding="same"),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.Conv2D(256, (1, 1), padding="same"),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.Conv2D(256, (1, 1), padding="same"),
      layers.Conv2D(512, (3, 3), padding="same"),
      layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
      layers.Flatten(),
      layers.Dropout(rate=0.5),
      layers.Dense(CLASS_NO, activation="softmax", name="output"),
    ])
    m.compile(
      loss=losses.CategoricalCrossentropy(),
      metrics=['accuracy'],
      optimizer=optimizers.Adam(learning_rate=0.00001),
    )
    return m

model=get_model()
model.fit(train_ds, epochs=EPOCHS, validation_data=validation_ds, verbose=1)
model.summary()
model.save(MODEL_OUTPUT_DIR)

358/358 [==============================] - 93s 221ms/step - loss: 3.1136 - accuracy: 0.0667 - val_loss: 3.0066 - val_accuracy: 0.1057
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 50, 180, 4)        0         
                                                                 
 conv2d (Conv2D)             (None, 50, 180, 32)       1184      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 25, 90, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 25, 90, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 45, 64)       0         
 2D)                                                  

Hiển thị thông tin các layers input/ouput

In [7]:
!saved_model_cli show --dir /content/drive/MyDrive/captcha/captcha-model --tag_set serve --signature_def serving_default

The given SavedModel SignatureDef contains the following input(s):
  inputs['rescaling_input'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 50, 180, 4)
      name: serving_default_rescaling_input:0
The given SavedModel SignatureDef contains the following output(s):
  outputs['output'] tensor_info:
      dtype: DT_FLOAT
      shape: (-1, 23)
      name: StatefulPartitionedCall:0
Method name is: tensorflow/serving/predict
